In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency ,kruskal , ttest_ind
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
joined_files = os.path.join("D:\\Data\\684_data", "divvy-tripdata_*")

In [3]:
joined_list = glob.glob(joined_files)

In [4]:
joined_list

['D:\\Data\\684_data\\divvy-tripdata_1.csv',
 'D:\\Data\\684_data\\divvy-tripdata_10.csv',
 'D:\\Data\\684_data\\divvy-tripdata_11.csv',
 'D:\\Data\\684_data\\divvy-tripdata_12.csv',
 'D:\\Data\\684_data\\divvy-tripdata_13.csv',
 'D:\\Data\\684_data\\divvy-tripdata_14.csv',
 'D:\\Data\\684_data\\divvy-tripdata_15.csv',
 'D:\\Data\\684_data\\divvy-tripdata_16.csv',
 'D:\\Data\\684_data\\divvy-tripdata_17.csv',
 'D:\\Data\\684_data\\divvy-tripdata_18.csv',
 'D:\\Data\\684_data\\divvy-tripdata_19.csv',
 'D:\\Data\\684_data\\divvy-tripdata_2.csv',
 'D:\\Data\\684_data\\divvy-tripdata_20.csv',
 'D:\\Data\\684_data\\divvy-tripdata_21.csv',
 'D:\\Data\\684_data\\divvy-tripdata_22.csv',
 'D:\\Data\\684_data\\divvy-tripdata_23.csv',
 'D:\\Data\\684_data\\divvy-tripdata_24.csv',
 'D:\\Data\\684_data\\divvy-tripdata_3.csv',
 'D:\\Data\\684_data\\divvy-tripdata_4.csv',
 'D:\\Data\\684_data\\divvy-tripdata_5.csv',
 'D:\\Data\\684_data\\divvy-tripdata_6.csv',
 'D:\\Data\\684_data\\divvy-tripdata_7.c

In [5]:
mydata = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [6]:
mydata.shape

(11262780, 13)

In [7]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11262780 entries, 0 to 11262779
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 1.1+ GB


In [8]:
round((mydata.isna().sum() / len(mydata))*100,2)

ride_id                0.00
rideable_type          0.00
started_at             0.00
ended_at               0.00
start_station_name    13.53
start_station_id      13.53
end_station_name      14.49
end_station_id        14.49
start_lat              0.00
start_lng              0.00
end_lat                0.09
end_lng                0.09
member_casual          0.00
dtype: float64

In [9]:
mydata.dropna(how='any',inplace=True)

In [10]:
mydata.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,1/13/2022 11:59,1/13/2022 12:02,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,1/10/2022 8:41,1/10/2022 8:46,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,1/25/2022 4:53,1/25/2022 4:58,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,1/4/2022 0:18,1/4/2022 0:33,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,1/20/2022 1:31,1/20/2022 1:37,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


In [11]:
def lowerS(x):
    return x.lower().strip()

mydata['start_station_name'] = mydata['start_station_name'].apply(lambda x : lowerS(x))
mydata['end_station_name'] = mydata['end_station_name'].apply(lambda x : lowerS(x))

In [12]:
mydata['started_at'] = pd.to_datetime(mydata['started_at'])
mydata['ended_at'] = pd.to_datetime(mydata['ended_at'])

mydata['Total_Duration'] = round((pd.to_datetime(mydata['ended_at']) - pd.to_datetime(mydata['started_at'])) / pd.Timedelta(minutes=1),2)

mydata['month'] = mydata['started_at'].dt.month

mydata['day'] = mydata['started_at'].dt.day

mydata['hour'] = mydata['started_at'].dt.hour

mydata['DayOfWeek'] = mydata['started_at'].dt.strftime('%A')

mydata['year'] = mydata['started_at'].dt.year

mydata['route']  = mydata['start_station_name'] +" to "+ mydata['end_station_name']

mydata['Quater'] = pd.to_datetime(mydata['started_at']).dt.quarter

In [13]:
mydata['Total_Duration'].describe()

count    8.957662e+06
mean     1.951054e+01
std      1.361806e+02
min     -1.687000e+02
25%      6.480000e+00
50%      1.138000e+01
75%      2.058000e+01
max      5.594415e+04
Name: Total_Duration, dtype: float64

In [14]:
false_record =  mydata[mydata['Total_Duration'] <= 0].index

mydata = mydata.drop(labels=false_record,axis=0) 

In [15]:
seasons = {(1, 12, 2): 'Winter', (3, 4, 5): 'Spring', (6, 7, 8): 'Summer', (9, 10, 11): 'Fall'}

def season(ser):
    for k in seasons.keys():
        if ser in k:
            return seasons[k]

mydata['season'] = mydata['started_at'].dt.month.apply(season)

In [16]:
Qua = {1:'Q1',2:'Q2',3:'Q3',4:'Q4'}

mydata['Quater'] = mydata['Quater'].replace(Qua)

In [17]:
mydata.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,Total_Duration,month,day,hour,DayOfWeek,year,route,Quater,season
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:00,2022-01-13 12:02:00,glenwood ave & touhy ave,525,clark st & touhy ave,RP-007,42.012800,-87.665906,...,casual,3.0,1,13,11,Thursday,2022,glenwood ave & touhy ave to clark st & touhy ave,Q1,Winter
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:00,2022-01-10 08:46:00,glenwood ave & touhy ave,525,clark st & touhy ave,RP-007,42.012763,-87.665967,...,casual,5.0,1,10,8,Monday,2022,glenwood ave & touhy ave to clark st & touhy ave,Q1,Winter
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:00,2022-01-25 04:58:00,sheffield ave & fullerton ave,TA1306000016,greenview ave & fullerton ave,TA1307000001,41.925602,-87.653708,...,member,5.0,1,25,4,Tuesday,2022,sheffield ave & fullerton ave to greenview ave...,Q1,Winter
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:00,2022-01-04 00:33:00,clark st & bryn mawr ave,KA1504000151,paulina st & montrose ave,TA1309000021,41.983593,-87.669154,...,casual,15.0,1,4,0,Tuesday,2022,clark st & bryn mawr ave to paulina st & montr...,Q1,Winter
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:00,2022-01-20 01:37:00,michigan ave & jackson blvd,TA1309000002,state st & randolph st,TA1305000029,41.877850,-87.624080,...,member,6.0,1,20,1,Thursday,2022,michigan ave & jackson blvd to state st & rand...,Q1,Winter


In [18]:
pd.crosstab(mydata['member_casual'],mydata['rideable_type'])

chi2_contingency(pd.crosstab(mydata['member_casual'],mydata['rideable_type']))

(739703.1454738233,
 0.0,
 2,
 array([[2481289.81933429,  206913.40023694, 1118061.78042876],
        [3357446.18066571,  279975.59976306, 1512855.21957124]]))

The p-value of the test, which is 0.0. A p-value of 0.0 suggests strong evidence against the null hypothesis, indicating that there is a significant association between the two variables.<br>
Overall, the code performs a chi-square test to determine if there is a relationship between the 'member_casual' and 'rideable_type' variables in the 'mydata' dataset, and the output provides statistical results and expected frequency values.

In [19]:
member_time = mydata[mydata['member_casual'] == 'member']['Total_Duration']
casual_time = mydata[mydata['member_casual'] == 'casual']['Total_Duration']


ttest_ind(member_time,casual_time,equal_var=False)

Ttest_indResult(statistic=-147.79941433975586, pvalue=0.0)

pvalue: This represents the p-value associated with the t-test. In the given output, the p-value is reported as 0.0, which indicates that the difference between the means of the two groups is statistically significant.<br>
In summary, the output of the t-test suggests a strong evidence of a significant difference between the means of the 'Total_Duration' for the 'member' and 'casual' groups in the 'mydata' dataset. The negative t-statistic indicates that the mean 'Total_Duration' for the 'member' group is lower than the mean 'Total_Duration' for the 'casual' group.

In [22]:
pd.crosstab([mydata['member_casual'],mydata['rideable_type']],[mydata['year'],mydata['Quater']],normalize='columns')

year                             2021                                    2022  \
Quater                             Q1        Q2        Q3        Q4        Q1   
member_casual rideable_type                                                     
casual        classic_bike   0.178128  0.280855  0.334262  0.171956  0.130456   
              docked_bike    0.057296  0.088207  0.069475  0.038593  0.027307   
              electric_bike  0.062431  0.103290  0.101242  0.123607  0.088725   
member        classic_bike   0.569053  0.423311  0.404435  0.453240  0.513764   
              docked_bike    0.000003  0.000000  0.000000  0.000000  0.000000   
              electric_bike  0.133090  0.104337  0.090586  0.212604  0.239747   

year                                                       
Quater                             Q2        Q3        Q4  
member_casual rideable_type                                
casual        classic_bike   0.245467  0.218193  0.132788  
              docked_bike    0.048955  0.042608  0.024910  
              electric_bike  0.138562  0.189349  0.160594  
member        classic_bike   0.396720  0.354997  0.402355  
              docked_bike    0.000000  0.000000  0.000000  
              electric_bike  0.170296  0.194853  0.279353